In [12]:
from sentence_transformers import SentenceTransformer, util
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import sqlparse

In [ ]:
def generate_candidate_sql(model_name, nl_query, db_schema, num_candidates=3):
    """
    Generates SQL queries for a given NL query and database schema using Chain of Thought reasoning.

    """
    # Load the tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    # CoT prompt
    prompt = f"""
Step 1: Understand the schema and identify the tables and columns needed for the query.
Step 2: Break down the natural language query into logical conditions.
Step 3: Translate the logical conditions into SQL clauses (SELECT, FROM, WHERE, etc.).
Step 4: Assemble the SQL query step by step.

Database Schema:
{db_schema}

Natural Language Query:
{nl_query}

Step-by-step reasoning and SQL query:
"""

    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)

    # Generate SQL candidates using Beam Search
    outputs = model.generate(
        inputs["input_ids"],
        max_length=300,
        num_beams=5,
        temperature=0.7,
        num_return_sequences=num_candidates
    )

    # Decode and return the generated SQL queries
    sql_candidates = [tokenizer.decode(output, skip_special_tokens=True).strip() for output in outputs]
    return sql_candidates

model_name = "google/flan-t5-large"
db_schema = """
Table Employee (emp_id, emp_name, dep_id, emp_salary)
Table Department (dep_id, dep_name)
"""
nl_query = "What are the names of employees in the HR department?"

candidates = generate_candidate_sql(model_name, nl_query, db_schema, num_candidates=3)

print("Generated SQL Candidates with Reasoning:")
for i, candidate in enumerate(candidates, 1):
    print(f"{i}. {candidate}")

Generated SQL Candidates with Reasoning:
1. SELECT T2.emp_name FROM department AS T1 JOIN employee AS T2 ON T1.dep_id = T2.dep_id WHERE T1.dep_name = "HR"
2. SELECT T2.emp_name FROM department AS T1 JOIN employee AS T2 ON T1.department_id = T2.department_id WHERE T1.department_name = "HR"
3. SELECT T1.emp_name , T1.emp_dept , T1.emp_salary FROM department AS T1 JOIN employee AS T2 ON T1.emp_id = T2.emp_id WHERE T2.dept_name = "HR"


In [19]:
def first_stage_ranking(nl_query, sql_candidates, top_k=5):
    """
    Coarse-grained ranking of SQL candidates based on semantic similarity.

    """
    model = SentenceTransformer('all-MiniLM-L6-v2')  # semantic model

    # Encode the natural language query and SQL candidates
    nl_embedding = model.encode(nl_query, convert_to_tensor=True)
    sql_embeddings = model.encode(sql_candidates, convert_to_tensor=True)

    # Compute cosine similarity
    scores = util.cos_sim(nl_embedding, sql_embeddings)[0]

    # Sort and select top-k candidates
    ranked_candidates = sorted(zip(sql_candidates, scores.tolist()), key=lambda x: x[1], reverse=True)
    return [candidate for candidate, score in ranked_candidates[:top_k]]


In [20]:
def second_stage_ranking(nl_query, sql_candidates, model_name="bert-base-uncased"):
    """
    Fine-grained ranking of SQL candidates using a pre-trained transformer model.

    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    # Compute scores for each candidate
    scores = []
    for sql_query in sql_candidates:
        inputs = tokenizer(nl_query, sql_query, return_tensors="pt", truncation=True, padding=True)
        outputs = model(**inputs)
        logits = outputs.logits[0]  # Extract logits
        positive_class_score = logits[1].item()  # Use the positive class for ranking
        scores.append(positive_class_score)

    # Rank candidates by score
    ranked_candidates = sorted(zip(sql_candidates, scores), key=lambda x: x[1], reverse=True)
    return ranked_candidates


In [21]:
def validate_sql_query(sql_query):
    try:
        parsed = sqlparse.parse(sql_query)
        return bool(parsed)  # Returns False if parsing fails
    except Exception as e:
        return False


In [22]:
def rank_and_select_sql_query(nl_query, sql_candidates):
    """
    Ranks SQL candidates and selects the best query.

    """
    # Step 1: First-stage ranking
    top_candidates = first_stage_ranking(nl_query, sql_candidates, top_k=5)

    # Step 2: Second-stage ranking
    ranked_candidates = second_stage_ranking(nl_query, top_candidates)

    # Step 3: Select the top-ranked query
    best_candidate, score = ranked_candidates[0]

    # Step 4: Validate the top query
    if validate_sql_query(best_candidate):
        return best_candidate
    else:
        return "No valid SQL query found."


In [24]:
# Input: Natural Language Query
nl_query = "What are the names of employees in the HR department?"

# Input: List of SQL Candidate Queries Generated using CoT
sql_candidates = [
    "SELECT T2.emp_name FROM department AS T1 JOIN employee AS T2 ON T1.dep_id = T2.dep_id WHERE T1.dep_name = 'HR';",
    "SELECT T2.emp_name FROM department AS T1 JOIN employee AS T2 ON T1.department_id = T2.department_id WHERE T1.department_name = 'HR';",
    "SELECT T1.emp_name , T1.emp_dept , T1.emp_salary FROM department AS T1 JOIN employee AS T2 ON T1.emp_id = T2.emp_id WHERE T2.dept_name = 'HR';"
]

# Rank and select the best SQL query
best_sql_query = rank_and_select_sql_query(nl_query, sql_candidates)

print("Best SQL Query:")
print(best_sql_query)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Best SQL Query:
SELECT T2.emp_name FROM department AS T1 JOIN employee AS T2 ON T1.dep_id = T2.dep_id WHERE T1.dep_name = 'HR';
